In [ ]:
import os
import sys
sys.path.append('./')
sys.path.append('../')
import numpy as np

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from art.estimators.classification import SklearnClassifier
from art.attacks.evasion import FastGradientMethod

from Core.dataset import CDataset

# Step 1: Load dataset

## Wine Dataset

In [ ]:
data = load_wine()
X, y = data.data, data.target

# Standardize features for stability
scaler = StandardScaler()
X = scaler.fit_transform(X)

print(np.unique(y, return_counts=True))


## Kyoto

In [ ]:
cDATA = 'Kyoto'

objDS = CDataset()

strFileName =  r'../../DATA/Kyoto2015DS.csv'
#strPath = os.path.join(strRoot, strFileName)
X, y = objDS.GetKyotoDataset(strFileName)
print('Kyoto:', X.shape, y.shape)

fTestSize = 0.9

## Spilt Data

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=fTestSize, 
                                                    random_state=42)

In [ ]:
print('Training Data:', X_train.shape, y_train.shape)
print('Testing Data:', X_test.shape, y_test.shape)

print(np.unique(y_train, return_counts=True))
print(np.unique(y_test, return_counts=True))

# Step 2: Train a simple classifier

## Train

In [ ]:
model = RandomForestClassifier()
#model = SVC(C=1.0, kernel="rbf")
model.fit(X_train, y_train)

## Test

In [ ]:
# Evaluate on clean test set
acc_clean = model.score(X_test, y_test)
print(f"Accuracy on clean test set: {acc_clean:.4f}")

# Step 3: Wrap model for ART

In [ ]:
classifier = SklearnClassifier(model=model)

In [ ]:
classifier.fit(X_train, y_train)

# Step 4: Create adversarial attack (FGSM)

In [ ]:
attack = FastGradientMethod(estimator=classifier, eps=0.1)

# Step 5: Generate adversarial examples

In [ ]:
X_test_adv = attack.generate(x=X_test)

# Step 6: Evaluate on adversarial set

In [ ]:
acc_adv = model.score(X_test_adv, y_test)
print(f"Accuracy on adversarial test set: {acc_adv:.4f}")

# Optional: Compare a few original vs adversarial samples
print("\nOriginal sample (first row):", X_test[0])
print("Adversarial sample (first row):", X_test_adv[0])